#**SQL Hotel Project** 
#####Visualizations and interpretation --> [google docs link](https://docs.google.com/presentation/d/1BxkntIwNDQMof5EpFoGdG6GScUyCtYCkaqJWFJ3Ohlc/edit?usp=sharing )


In [ ]:
''' 
The dataset contains five tables:

1. hotel_2018 -- relevant columns: hotel, arrival_date_year, arrival_date_month, arrival_date_day_of_month, stays_in_week_nights,
                                    stays_in_week_nights, adults, children, adr, market_segment, meal, is_canceled, required_car_parking_spaces

2. hotel_2019 -- relevant columns: hotel, arrival_date_year, arrival_date_month, arrival_date_day_of_month, stays_in_week_nights,
                                    stays_in_week_nights, adults, children, adr, market_segment, meal, is_canceled, required_car_parking_spaces

3. hotel_2020 -- relevant columns: hotel, arrival_date_year, arrival_date_month, arrival_date_day_of_month, stays_in_week_nights,
                                    stays_in_week_nights, adults, children, adr, market_segment, meal, is_canceled, required_car_parking_spaces
4. meal -- columns: meal,cost

5. market_segment -- columns: market_segment, Discount
 
Each row in hotel_2018, hotel_2019, hotel_2020 represents a booking. We have two type of hotels: City Hotel and Resort Hotel

In [ ]:
### Query 1: Does the revenue for each hotel type grow by month? ###

# to get a single date column in DATE-format we concatonate the three columns for year, month and day --> extract year and month for our calculation
# to be able to calcualte our revenue we have to JOIN the hotel table for each year with the meal and market_segment table
# revenue calculation includes: adr (=average daily rate), booked nights, number of persons, meal cost, Discount
# to get the hotel data for all 3 years we have to perform a FULL UNION between the three hotel tables

with revenue_table as (SELECT 
    h1.hotel, DATE_FORMAT(STR_TO_DATE(concat(arrival_date_year,' ',arrival_date_month,' ',arrival_date_day_of_month), '%Y %M %D'), '%Y-%m')
    AS Arrival, SUM((h1.adr*(h1.stays_in_weekend_nights) + h1.adr*(h1.stays_in_week_nights)) + ((mt.cost*(h1.stays_in_weekend_nights)
    + mt.cost*(h1.stays_in_week_nights))*(h1.adults + h1.children))*(1-tm.Discount)) as revenue
    from hotel.hotel_2018 h1    
  left join hotel.table_market tm
     on h1.market_segment = tm.market_segment
  left join hotel.meal_table mt
     on h1.meal = mt.meal
 where h1.is_canceled = 0               # we don't include canceled bookings
 GROUP BY 1,2   
    
  UNION ALL 
    Select h2.hotel, DATE_FORMAT(STR_TO_DATE(concat(arrival_date_year,' ',arrival_date_month,' ',arrival_date_day_of_month), '%Y %M %D'), '%Y-%m')
    AS Arrival,SUM((h2.adr*(h2.stays_in_weekend_nights) + h2.adr*(h2.stays_in_week_nights)) + ((mt.cost*(h2.stays_in_weekend_nights)
    + mt.cost*(h2.stays_in_week_nights))*(h2.adults + h2.children))*(1-tm.Discount)) as revenue
    from hotel.hotel_2019 h2    
  left join hotel.table_market tm
    on h2.market_segment = tm.market_segment
  left join hotel.meal_table mt
    on h2.meal = mt.meal
 where h2.is_canceled = 0
 GROUP BY 1, 2

  UNION ALL 
    Select h3.hotel, DATE_FORMAT(STR_TO_DATE(concat(arrival_date_year,' ',arrival_date_month,' ',arrival_date_day_of_month), '%Y %M %D'), '%Y-%m')
    AS Arrival, 
    SUM((h3.adr*(h3.stays_in_weekend_nights) + h3.adr*(h3.stays_in_week_nights)) + ((mt.cost*(h3.stays_in_weekend_nights)
    + mt.cost*(h3.stays_in_week_nights))*(h3.adults + h3.children))*(1-tm.Discount)) as revenue
    from hotel.hotel_2020 h3 
  left join hotel.table_market tm
    on h3.market_segment = tm.market_segment
  left join hotel.meal_table mt
    on h3.meal = mt.meal
 where h3.is_canceled = 0
 GROUP BY 1, 2
),

# with a WINDOW FUNCTION we can display the monthly revenue in relation to the previous month's revenue
# we do the same with the previous year's month to display the revenue change over a year  

lag_table_month as(Select*,  LAG(e.revenue, 1,0) over (partition by e.hotel order by e.Arrival) as previous_month from revenue_table e),
lag_table_year as(Select*,  LAG(e.revenue, 12,0) over (partition by e.hotel order by e.Arrival) as previous_year from revenue_table e),

# we calcualte the growth percentage change over a month and year respectively and store the result in a CTE 

another_table as (Select x.hotel, x.Arrival, x.revenue, x.Growth_percentage_month, x.Growth_percentage_year from 
(Select lm.*, Cast((((lm.revenue-previous_month)/previous_month)*100) as Decimal (5,2))
 as Growth_percentage_month, Cast((((ly.revenue-previous_year)/previous_year)*100) as Decimal (5,2))
 as Growth_percentage_year from lag_table_month lm join lag_table_year ly on lm.Arrival = ly.Arrival and lm.hotel = ly.hotel)x)

# with an SELF JOIN we pivot the hotel type table and create seperate growth columns for each type
# we also calculate the difference in grwoth between the two hotel types

Select t1.Arrival as Date, coalesce(t1.Growth_percentage_month, '') as 'Resort Hotel Growth in % - previous Month',
  t2.Growth_percentage_month as 'City Hotel Growth in % - previous Month',
  t1.Growth_percentage_month-t2.Growth_percentage_month as 'Difference in Growth - Month',
  coalesce(t1.Growth_percentage_year, '') as 'Resort Hotel Growth in % - previous Year',
  coalesce(t2.Growth_percentage_year,'') as 'City Hotel Growth in % - previous Year',
  coalesce(t1.Growth_percentage_year-t2.Growth_percentage_year,'') as 'Difference in Growth - Year'
  from another_table t1
inner join another_table t2 on t1.Arrival = t2.Arrival and t1.hotel > t2.hotel
  where t1.Growth_percentage_month != '' order by t1.Arrival ;



In [ ]:
### Query 2: Do the hotels need more parking space? ###

# We proceed similar like in #Query 1 and perform FULL UNION on the three tables and store the result in a CTE
# with the aggregate function SUM we calcuate the number of requested parking spaces

with parking_table as (SELECT 
    h1.hotel, DATE_FORMAT(STR_TO_DATE(concat(arrival_date_year,' ',arrival_date_month,' ',arrival_date_day_of_month), '%Y %M %D'), '%Y-%m') AS Arrival,
    SUM(h1.required_car_parking_spaces) as Parking
   
FROM
    hotel.hotel_2018 h1  
    where h1.is_canceled = 0
    GROUP BY 1,2 
    
UNION ALL 
 Select h2.hotel, DATE_FORMAT(STR_TO_DATE(concat(arrival_date_year,' ',arrival_date_month,' ',arrival_date_day_of_month), '%Y %M %D'), '%Y-%m') AS Arrival,
 SUM(h2.required_car_parking_spaces) as Parking  
   
FROM
hotel.hotel_2019 h2  
where h2.is_canceled = 0
GROUP BY 1,2

UNION ALL 
 Select h3.hotel, DATE_FORMAT(STR_TO_DATE(concat(arrival_date_year,' ',arrival_date_month,' ',arrival_date_day_of_month), '%Y %M %D'), '%Y-%m') AS Arrival,
     SUM(h3.required_car_parking_spaces) as Parking
   
FROM
hotel.hotel_2020 h3  
where h3.is_canceled = 0
GROUP BY 1,2
),

# with a WINDOW FUNCTION we can see the change of parking requests to the previous year by month

lag_table as(Select*,  LAG(p.Parking, 12,0) over (partition by p.hotel order by p.arrival) as previous_month from parking_table p),

# we calculate the growth change

another_table as (Select x.hotel, x.arrival, x.parking,
 x.Growth_percentage from (Select l.*, Cast((((l.parking-previous_month)/previous_month)*100) as Decimal (7,2))
 as Growth_percentage from lag_table l)x)
 
# we perform an INNER JOIN again to pivot the hotel type column and display the growth change 

Select t1.arrival, coalesce(t1.Growth_percentage, '') as 'Resort Hotel Parking Request Growth in % - previous Year',
t2.Growth_percentage as 'City Hotel Parking Request Growth in % - previous Year' 
from another_table t1
inner join another_table t2 on t1.arrival = t2.arrival and t1.hotel > t2.hotel where t1.Growth_percentage != '' order by t1.arrival ;


In [ ]:
### Query 3: How does the ADR (average daily rate) change over time and seasons? In which season do we have the highest rate? ###

# We proceed similar like in #Query 1 and #Query 2 and perform FULL UNION on the three tables and store the result in a CTE

With date_table as(Select h1.hotel, Month(STR_TO_DATE(h1.arrival_date_month, '%MM')) AS Month,h1.arrival_date_year as Year,
  avg(h1.adr) as ADR
from hotel.hotel_2018 h1 
where h1.is_canceled = 0 
Group by 1,2,3

UNION ALL
Select h2.hotel, Month(STR_TO_DATE(h2.arrival_date_month, '%MM')) AS Month,h2.arrival_date_year as Year,
  avg(h2.adr) as ADR 
from hotel.hotel_2019 h2  
where h2.is_canceled = 0 
Group by 1,2,3

UNION ALL
Select h3.hotel, Month(STR_TO_DATE(h3.arrival_date_month, '%MM')) AS Month,h3.arrival_date_year as Year,
  avg(h3.adr) as ADR 
from hotel.hotel_2020 h3  
where h3.is_canceled = 0 
Group by 1,2,3
),

# with CASE WHEN we pivot the calculated average ADR for each season
# which values are assigned to wich season is dependent on the month number
# as we want to look at the ADR development over the seasons by year we group by year

season_table as (Select dt.hotel, dt.year, 
cast(AVG(Case when dt.Month in (6,7,8) then ADR else null end) as Decimal(7,2)) as ADR_Summer,
coalesce(cast(AVG(Case when dt.Month in (9,10,11) then ADR else null end) as Decimal(7,2)),'No Data') as ADR_Autumn,
cast(AVG(Case when dt.Month in (12,1,2) then ADR else null end) as Decimal(7,2)) as ADR_Winter,
coalesce(cast(AVG(Case when dt.Month in (3,4,5) then ADR else null end) as Decimal(7,2)),'No Data') as ADR_Spring
from date_table dt group by 1,2 order by 1),

# with a WINDOW FUNCTION we compare the ADR of one season to the previous years's same season

lag_table as(Select*,  LAG(st.ADR_Summer, 1,0) over (partition by st.hotel order by st.hotel, st.year) as previous_summer,
 LAG(st.ADR_Autumn, 1,0) over (partition by st.hotel order by st.hotel, st.year) as previous_autumn,
 LAG(st.ADR_Winter, 1,0) over (partition by st.hotel order by st.hotel, st.year) as previous_winter,
 LAG(st.ADR_Spring, 1,0) over (partition by st.hotel order by st.hotel, st.year) as previous_spring
 from season_table st),

# this CTE return the yearly growth change of the seasons

growth_table as (Select lt.hotel, lt.year, lt.ADR_Summer, coalesce(cast((((ADR_Summer-previous_summer)/previous_summer)*100)
as Decimal(5,2)),'No Data') as 'Growth % - previous Summer',
lt.ADR_Autumn, coalesce(cast((((ADR_Autumn-previous_autumn)/previous_autumn)*100)as Decimal(5,2)),'No Data') as 'Growth % - previous Autumn',
lt.ADR_Winter, coalesce(cast((((ADR_Winter-previous_winter)/previous_winter)*100)as Decimal(5,2)),'No Data') as 'Growth % - previous Winter',
lt.ADR_Spring, coalesce(cast((((ADR_Spring-previous_spring)/previous_spring)*100)as Decimal(5,2)),'No Data') as 'Growth % - previous Spring'
from lag_table lt)

Select gt.* from growth_table gt;



In [ ]:
### Query 4: How does the volume of guests change over time and seasons? In which season do we have the highest guest vulume? ###


#We proceed similar like in the previous queries and perform FULL UNION on the three tables and store the result in a CTE

With date_table as(Select h1.hotel, Month(STR_TO_DATE(h1.arrival_date_month, '%MM')) AS Month,h1.arrival_date_year as Year,
  sum(h1.adults + h1.children) as AVG_Guest from hotel.hotel_2018 h1 where h1.is_canceled = 0 Group by 1,2,3

UNION ALL
Select h2.hotel, Month(STR_TO_DATE(h2.arrival_date_month, '%MM')) AS Month,h2.arrival_date_year as Year, sum(h2.adults + h2.children)
 from hotel.hotel_2019 h2  where h2.is_canceled = 0 Group by 1,2,3

UNION ALL
Select h3.hotel, Month(STR_TO_DATE(h3.arrival_date_month, '%MM')) AS Month,h3.arrival_date_year as Year, sum(h3.adults + h3.children)
 from hotel.hotel_2020 h3  where h3.is_canceled = 0 Group by 1,2,3),

# with CASE WHEN we pivot the calculated average guest volume for each season
# which values are assigned to wich season is dependent on the month number
# as we want to look at the guest volume development over the seasons by year we group by year

season_table as (Select dt.hotel, dt.year, 
cast(AVG(Case when dt.Month in (6,7,8) then  AVG_Guest else null end) as Decimal(7,2)) as  AVG_Guest_Summer,
coalesce(cast(AVG(Case when dt.Month in (9,10,11) then  AVG_Guest else null end) as Decimal(7,2)),'No Data') as  AVG_Guest_Autumn,
cast(AVG(Case when dt.Month in (12,1,2) then  AVG_Guest else null end) as Decimal(7,2)) as  AVG_Guest_Winter,
coalesce(cast(AVG(Case when dt.Month in (3,4,5) then  AVG_Guest else null end) as Decimal(7,2)),'No Data') as  AVG_Guest_Spring
from date_table dt group by 1,2 order by 1),

# with a WINDOW FUNCTION we compare the guest volume of one season to the previous years's same season

lag_table as(Select*,  LAG(st. AVG_Guest_Summer, 1,0) over (partition by st.hotel order by st.hotel, st.year) as previous_summer,
 LAG(st. AVG_Guest_Autumn, 1,0) over (partition by st.hotel order by st.hotel, st.year) as previous_autumn,
 LAG(st. AVG_Guest_Winter, 1,0) over (partition by st.hotel order by st.hotel, st.year) as previous_winter,
 LAG(st. AVG_Guest_Spring, 1,0) over (partition by st.hotel order by st.hotel, st.year) as previous_spring
 from season_table st),

# this CTE return the yearly growth change of the seasons

growth_table as (Select lt.hotel, lt.year, lt. AVG_Guest_Summer, coalesce(cast(((( AVG_Guest_Summer-previous_summer)/previous_summer)*100)
as Decimal(5,2)),'No Data') as 'Growth % - previous Summer',
lt.AVG_Guest_Autumn, coalesce(cast((((AVG_Guest_Autumn-previous_autumn)/previous_autumn)*100)as Decimal(5,2)),'No Data')
 as 'Growth % - previous Autumn',
lt.AVG_Guest_Winter, coalesce(cast((((AVG_Guest_Winter-previous_winter)/previous_winter)*100)as Decimal(5,2)),'No Data')
 as 'Growth % - previous Winter',
lt.AVG_Guest_Spring, coalesce(cast((((AVG_Guest_Spring-previous_spring)/previous_spring)*100)as Decimal(5,2)),'No Data')
 as 'Growth % - previous Spring'
from lag_table lt)

Select gt.* from growth_table gt;
